In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
from random import randint
import math
from numpy.linalg import inv

def takeAvg(total,size):#To use in DLT
    avg=total/float(size)
    return avg
def totalize(points,axis):#To use in DLT
    total=0
    for i in range(len(points)):
        total+=float(points[i][axis])
        
    return total
def normalize(points,avgx,avgy):#To use in DLT
    newPoints=[]
    for i in range(len(points)):
        normalized_x=float(points[i][0])-avgx
        normalized_y=float(points[i][1])-avgy
        newPoints.append([normalized_x,normalized_y])
        
    return newPoints
def scale(points):#To use in DLT
    distance=0
    for i in range(len(points)):
        distance += float(np.sqrt(np.square(points[i][0]) + np.square(points[i][1])))
    scale=  np.sqrt(2)/float(distance)
    return scale
def detect_keyPoints(img1,img2):
    sift = cv2.xfeatures2d.SIFT_create(contrastThreshold = 0.1 ,edgeThreshold = 10)
    kp1, des1 = sift.detectAndCompute(img1,None)
    kp2, des2 = sift.detectAndCompute(img2,None)
    newList1 = []
    newList2 = []
    temp=0
    for i in range(len(des1)):
        distance=1000000000

        temp=0
        for y in range(len(des2)):
            x=0
            for a in range(len(des1[0])):
                x+=math.sqrt((des1[i][a]-des2[y][a])**2)

            if(x<distance):
                distance=x
                temp=int(y)
        newList1.append(kp1[i].pt)
        newList2.append(kp2[temp].pt)
        
    return newList1,newList2

def dlt(ref_points,dst_points):
    
    total_x=totalize(ref_points,0)
    total_y=totalize(ref_points,1)
    total_xprm=totalize(dst_points,0)
    totaly_yprm=totalize(dst_points,1)
    
    avg_x=takeAvg(total_x,len(ref_points))
    avg_y=takeAvg(total_y,len(ref_points))
    avg_xprm=takeAvg(total_xprm,len(dst_points))
    avg_yprm=takeAvg(totaly_yprm,len(dst_points))

    new_refPoints=normalize(ref_points,avg_x,avg_y)
    new_dstPoints=normalize(dst_points,avg_xprm,avg_yprm)
    
    ref_scale=scale(new_refPoints)
    dst_scale=scale(new_dstPoints)
    
    t=np.array([[float(ref_scale),0,float(ref_scale)*(-avg_x)]
                ,[0,float(ref_scale),float(ref_scale)*(-avg_y)],
                 [0,0,1]])
        
    t2=np.array([[float(dst_scale),0,float(dst_scale)*(-avg_xprm)],
                 [0,float(dst_scale),float(dst_scale)*(-avg_yprm)],
                 [0,0,1]])
    
    for i in range(len(new_refPoints)):
        nPoints=np.dot(t,np.array([[float(ref_points[i][0])],[float(ref_points[i][1])],[1]]))
        new_refPoints[i][0]= nPoints[0]
        new_refPoints[i][1]= nPoints[1]
     
    for i in range(len(new_dstPoints)):
        nPoints2=np.dot(t2,np.array([[float(dst_points[i][0])],[float(dst_points[i][1])],[1]]))
        new_dstPoints[i][0]= nPoints2[0]
        new_dstPoints[i][1]= nPoints2[1]
        
    A_matrix=[]
    
    for i in range(len(new_refPoints)):
        A_matrix.append([0,0,0,-new_refPoints[i][0],-new_refPoints[i][1],-1,new_dstPoints[i][1]*new_refPoints[i][0],new_dstPoints[i][1]*new_refPoints[i][1],new_dstPoints[i][1]])
        A_matrix.append([new_refPoints[i][0],new_refPoints[i][1],1,0,0,0,-new_dstPoints[i][0]*new_refPoints[i][0],-new_refPoints[i][1]*new_dstPoints[i][0],-new_dstPoints[i][0]])

    A_matrix=np.array(A_matrix)
    w,u,vt=cv2.SVDecomp(np.asarray(A_matrix))
    H̃=[[vt[-1][0],vt[-1][1],vt[-1][2]],[vt[-1][3],vt[-1][4],vt[-1][5]],[vt[-1][6],vt[-1][7],vt[-1][8]]]
    invT = inv(t2)   
    tempH=np.dot(invT,H̃)
    H=np.dot(tempH,t)
    return H


def findInlier(H,keyPoints1,keyPoints2):
    current_refPoints=[]
    current_dstPoints=[]
    temp=0
    counter=0
    for x in range(len(keyPoints1)):
        distance=3.0
        temp=0
        y=0
        queryList=np.dot(H,np.array([[keyPoints1[x][0]],[keyPoints1[x][1]],[1]]))
        queryList = np.array(queryList)
        queryList[0]=queryList[0]/queryList[2]
        queryList[1]=queryList[1]/queryList[2]
        flag = False
        
        for y in  range (len(keyPoints2)):
            d=float(np.sqrt(np.square((queryList[0]-keyPoints2[y][0]))+(np.square(queryList[1]-keyPoints2[y][1]))))
            if (d<=3.0):

                if (distance>d):
                    distance=d
                    temp=int(y)
                    flag = True 

        if (flag == True):
            current_refPoints.append((keyPoints1[x]))
            current_dstPoints.append((keyPoints2[temp]))
            temp=0
            
    return current_refPoints,current_dstPoints

img1 = cv2.imread('img1.jpg',0) 
img2 = cv2.imread('img2.jpg',0)
keyPoint1,keyPoint2=detect_keyPoints(img1,img2)

keyPoint1=np.array(keyPoint1)
keyPoint2=np.array(keyPoint2)

N=10000
counter=0
prev_numberOfinliers=0
while(counter<N):
    
    numberOfinliers=0
    random=np.random.randint(0,len(keyPoint1))
    random2=np.random.randint(0,len(keyPoint1))
    random3=np.random.randint(0,len(keyPoint1))
    random4=np.random.randint(0,len(keyPoint1))
    
    refPoints=[(keyPoint1[random]),(keyPoint1[random2]),(keyPoint1[random3]),(keyPoint1[random4])]
    dstPoints=[(keyPoint2[random]),(keyPoint2[random2]),(keyPoint2[random3]),(keyPoint2[random4])]
  
    refPoints=np.array(refPoints)
    dstPoints=np.array(dstPoints)
  
    H=dlt(refPoints,dstPoints)
    
    for i in range(len(keyPoint1)):
        queryPoints=np.dot(H,np.array([[keyPoint1[i][0]],[keyPoint1[i][1]],[1]]))
    
        queryPoints[0]=queryPoints[0]/queryPoints[2]
        queryPoints[1]=queryPoints[1]/queryPoints[2]
        queryPoints=np.array(queryPoints)
        
        for y in range (len(keyPoint2)):
            
            if (np.sqrt(np.square((queryPoints[0]-keyPoint2[y][0]))+(np.square(queryPoints[1]-keyPoint2[y][1])))<=3.0):
                numberOfinliers+=1
                break
                
    ratio=float(numberOfinliers)/len(keyPoint1)
    
    if(numberOfinliers>prev_numberOfinliers):
            Homography=np.copy(H)
            max_numOfinliers=numberOfinliers
            N=int((-2.0/float(math.log10(1.0-(float(ratio**4))))) - 1.0)
            counter=0
            
    counter+=1
    prev_numberOfinliers=max_numOfinliers
    print ("RANSAC",N)
    print ("Number of inlier is",max_numOfinliers)

print ("INLIER RATIO:",(max_numOfinliers/len(keyPoint1))*100)
newRef=[]
newDst=[]
newRefCopy=[]
newDstCopy=[]

newRef,newDst=findInlier(Homography,keyPoint1,keyPoint2)

n=0
while(1):
  
    betterHomo=dlt(newRef,newDst)
    n+=1
    newRefCopy=np.copy(newRef)
    newDstCopy=np.copy(newDst)

    newRef=[]
    newDst=[]
    if (len(newRefCopy) <= 4 or  len(newDstCopy) <= 4):
        break
    newRef,newDst=findInlier(betterHomo,newRefCopy,newDstCopy)
    
    
    if (len(newRef)==len(newRefCopy)):
        break

leftCoord=np.dot(betterHomo,np.array([[154.0],[161.0],[1.0]]))
leftCoord[0]=float(leftCoord[0]/leftCoord[2])
leftCoord[1]=float(leftCoord[1]/leftCoord[2])

rightCoord=np.dot(betterHomo,np.array([[470.0],[161.0],[1.0]]))
rightCoord[0]=(rightCoord[0]/rightCoord[2])
rightCoord[1]=(rightCoord[1]/rightCoord[2])

bottomleftCoord=np.dot(betterHomo,np.array([[172.0],[576.0],[1.0]]))
bottomleftCoord[0]=(bottomleftCoord[0]/bottomleftCoord[2])
bottomleftCoord[1]=(bottomleftCoord[1]/bottomleftCoord[2])

bottomrightCoord=np.dot(betterHomo,np.array([[455.0],[576.0],[1.0]]))
bottomrightCoord[0]=float(bottomrightCoord[0]/bottomrightCoord[2])
bottomrightCoord[1]=float(bottomrightCoord[1]/bottomrightCoord[2])

img4 = cv2.imread('img2.jpg')

cv2.rectangle(img4,(leftCoord[0],leftCoord[1]),(bottomrightCoord[0],bottomrightCoord[1]),(0,0,255),5)
cv2.imwrite("object_detection.png",img4)


COMMENT-1:
-ORB is a very fast binary descriptor based on BRIEF which is rotation invariant and resistant to noise. ORB is at two orders of magnitude faster than SIFT, while performing as well in many
situations. The efficiency is tested on several real-world applications.
-Scale-Invariant Feature  Transform  (SIFT)  that  has  similar  matching  performance,  is  less  affected  by  image  noise,  and  is  capable of  being  used  for  real-time  performance.
-ORB  builds  on the FAST keypoint detector and  the  binary  BRIEF  descriptor with many modifications to enhance the performance. It uses FAST to find multiscale keypoints on several pyramid levels and applies a Harris corner measure to pick the best keypoints. To achieve rotation invariance, the orientation of the keypoint is calculated by using the intensity-weighted centroid of a circular patch with the
located  keypoint  at  the  centre. 

NOTE:IN that comment i searched information in this site: http://www.willowgarage.com/sites/default/files/orb_final.pdf

COMMENT-2:
-We can find the nearest neighbour with calculating distance of each keypoint of reference image with all keypoints of the query image and then we can apply an algorithm to find minimum of them and take it our nearest neighbour.
-For SIFT alorithm we calculate Euclidean distance
-For ORB alorithm we calculate Hamming distance

COMMENT-3:
-The input of the RANSAC algorithm is a set of data points which contains outliers. The goal is to find a model describing inliers from the given data set.
-We give a set of points as an input to RANSAC algorithm it contains some outliers and some inliers.Our purpose on this algorithm is to find inliers.So setting the Euclidean distance smaller than 3 pixels 
is good because greather outliers will be eliminated and this produce good result.

COMMENT-4:
-When we normalize points we  puts both points on the border which makes the solution more stable and puts it further from the center.So it provide us simplicity about estimating. By normalizing the data set, we center the data and give it unit variance. These conditions are better handled.This normalization affects to result of SVD computation.When we normalize we get better ratios between values.And it decrease noise to get exact solution.